# Problem

Count the number of distinct elements seen so far in a stream.

## Example

* stream: [a, b, a, c, b, ...] -> 3

A naive solution would be to keep a list of all distinct elements seen so far. i.e. $O(|\text{Distinct Elements}|)$.

Can we do with less space by sacrificing accuracy?

# Fact 1: Order Statistics

Independent random variables
$$X_1, X_2, \ldots, X_n$$

$$Z = \max(X_1, X_2, \ldots, X_n)$$

What does the distribution of $Z$ look like?

$$f(z) = P(Z \leq z) = P(X_1 \leq z, X_2 \leq z, \ldots, X_n \leq z) = P(X_1 \leq z)P(X_2 \leq z)\ldots P(X_n \leq z) = F(X_i)^n$$

where $F(X_i)$ is the CDF of $X_i$. i.e. pdf of $Z$ is a multiple of the cdf of $X_i$.

# Fact 2: Hashes Are i.i.d. uniform random variables

If we hash a set of elements, the hash values are i.i.d. uniform random variables.

$$h \to \{0, 1, \ldots, m-1\}$$

# Fact 3: Bad distinct estimate

Using the above facts, we can estimate the number of distinct elements seen so far by hashing the elements and taking the maximum hash value seen so far.

Total number of elements in the stream:
$$N$$

Total number of distinct elements in the stream:
$$n$$

Normalized hash value:
$$\hat{h} = \frac{h}{m}$$

We keep the maximum (normalized) hash value seen so far:
$$\hat{Z} = \max(\hat{h}(x_1), \hat{h}(x_2), \ldots, \hat{h}(x_N))$$

The pdf for Z is:
$$f(z) = nz^{n-1}$$

[https://www.colorado.edu/amath/sites/default/files/attached-files/order_stats.pdf]

Therefore, the expected value of Z is:
$$E[Z] = \frac{n}{n+1}$$

Using the observed maximum hash value, we can estimate the number of distinct elements seen so far:

$$\hat{Z} \approx \frac{n}{n+1}$$

$$ n \approx \frac{\hat{Z}}{1 - \hat{Z}}$$


This is a bad estimate because if we have seen an element with really high hash value early on, we will overestimate the number of distinct elements. For example if we somehow see an element with normalized hash value 0.99 early on, we will estimate the number of distinct elements to be 100. The hash function is uniform so it is equally likely to see an element with normalized hash value 0.99 as it is to see an element with normalized hash value 0.01. Therefore, it is hard to distinguish between a stream with 100 distinct elements and a stream with 1 distinct element.


<img src="images/flajoletmartin.JPG" alt="UniformVSLongTailed" width="400"/>

## Fact 4: Flajolet-Martin Algorithm

The Flajolet-Martin Algorithm uses the the position of the least significant 1 bit in the hash value to generate a skewed distribution from a uniform distribution. Intuitively, this skewness will make the estimate of the distinct element count by making high hash values less likely than low hash values. In the example above, we would be less likely to see an element with normalized hash value 0.99 than an element with normalized hash value 0.01.

The hash function will produce a value in the range $[0, m-1]$ as a binary number. Then the position of the least significant 1 bit is computed. For example

| Hash Value | Binary | Least Significant 1 Bit Position |
|------------|--------|--------------------------|
| 0          | 0000   | 0                        |
| 1          | 0001   | 1                        |
| 2          | 0010   | 2                        |
| 3          | 0011   | 1                        |
| 4          | 0100   | 3                        |
| 5          | 0101   | 1                        |
| 6          | 0110   | 2                        |
| 7          | 0111   | 1                        |


Now, $Prob(\text{Least Significant 1 Bit Position} = k) = \frac{1}{2^k}$. This is how the Flajolet-Martin Algorithm generates a skewed distribution from a uniform distribution.

We keep the maximum least significant 1 bit position seen so far:
$$\hat{Z} = \max(\text{Least Significant 1 Bit Position}(x_1), \text{Least Significant 1 Bit Position}(x_2), \ldots, \text{Least Significant 1 Bit Position}(x_N))$$

Then using that value $R = \hat{Z}$, we can estimate the number of distinct elements seen so far as:

$$n \approx \frac{2^{R + 1}}{\phi}$$

where $\phi \approx 0.77351$ is the Euler-Mascheroni constant.

Read more on: [https://cs328-2022.github.io/CS328-Notes/notebooks/2022_04_04_Flajolet_Martin_Algorithm.html]
